Keras Tuner tutorial

In [53]:
import pandas as pd
from tensorflow import keras
from keras import layers
from keras_tuner.tuners import RandomSearch

In [54]:
df = pd.read_csv("Real_Combine.csv")
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [69]:
df = df.dropna()
df.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    0
dtype: int64

In [70]:
X = df.iloc[:,0:7] # independent features
y = df["PM 2.5"] # dependent features

1.How many number of hidden layers should we have?
2.How many number of neurons in each layers?
3.Learning rate

In [71]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [73]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

In [74]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [75]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [76]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 07s]
val_mean_absolute_error: 46.377455393473305

Best val_mean_absolute_error So Far: 46.377455393473305
Total elapsed time: 00h 00m 40s
INFO:tensorflow:Oracle triggered exit


In [77]:
tuner.results_summary()

Results summary
Results in project\Air Quality Index
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 4 summary
Hyperparameters:
num_layers: 5
units_0: 224
units_1: 64
learning_rate: 0.01
units_2: 192
units_3: 416
units_4: 160
units_5: 288
units_6: 416
units_7: 224
units_8: 160
units_9: 128
units_10: 64
units_11: 32
units_12: 192
units_13: 128
units_14: 160
units_15: 480
units_16: 384
Score: 46.377455393473305

Trial 0 summary
Hyperparameters:
num_layers: 9
units_0: 512
units_1: 192
learning_rate: 0.01
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
Score: 46.72504552205404

Trial 1 summary
Hyperparameters:
num_layers: 4
units_0: 480
units_1: 32
learning_rate: 0.01
units_2: 256
units_3: 448
units_4: 128
units_5: 64
units_6: 32
units_7: 224
units_8: 352
Score: 47.45553716023763

Trial 2 summary
Hyperparameters:
num_layers: 17
units_0: 448
units_1: 96
learning_rate: 0.01
units_2: 160
units_3: 448
units_4: 224
uni